In [1]:
import time
import torch.nn as nn
from torch import optim
import numpy as np

from data_dev import Seq2SeqDataManager, to_padded_tensor, normalize_string, TOK_XX
from masked_cross_entropy import *
from decoder import LuongAttnDecoderRNN
from encoder import EncoderRNN
from utils import print_summary

from fit_dev import fit, predict

USE_CUDA = False
#PAD_token = 1
#SOS_token = 1
#SOS_token = 2
#EOS_token = 2
#EOS_token = 3
MIN_LENGTH = 3
#MAX_LENGTH = 25
MAX_LENGTH = 6
MIN_COUNT = 5



kept 1680 sequences from 1680 sequences
kept 1680 sequences from 1680 sequences
kept 1593 sequences from 1680 sequences
kept 1593 sequences from 1680 sequences
kept 94 sequences from 184 sequences
kept 94 sequences from 184 sequences
kept 34 sequences from 94 sequences
kept 34 sequences from 94 sequences


In [2]:
data_manager=Seq2SeqDataManager.create_from_txt('data/eng-fra_sub.txt', min_freq=MIN_COUNT, min_ntoks=MIN_LENGTH)
#data_manager=Seq2SeqDataManager.create_from_txt('data/eng-fra.txt', min_ntoks=3, max_ntoks=10)
##test
train_batch_size = 100
valid_batch_size=100
train_dataloader, valid_dataloader=data_manager.get_dataloaders(train_batch_size=train_batch_size, valid_batch_size=valid_batch_size)


##model conf
# Configure models
attn_model = 'dot'
#hidden_size = 500
hidden_size = 50
n_layers = 2
dropout = 0.1
#batch_size = 100


# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 0.5
#learning_rate = 0.0001
learning_rate = 0.001
decoder_learning_ratio = 5.0
# n_epochs = 50000
n_epochs =20
epoch = 0
# plot_every = 20
plot_every = 2000
# print_every = 100

# evaluate_every = 1000
evaluate_every = 1

# Initialize models
encoder = EncoderRNN(len(data_manager.train_seq2seq.seq_x.vocab.itos), hidden_size, n_layers, dropout=dropout)
decoder = LuongAttnDecoderRNN(attn_model, hidden_size, len(data_manager.train_seq2seq.seq_y.vocab.itos), USE_CUDA, n_layers, dropout=dropout)

# Initialize optimizers and criterion
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
#decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

kept 1593 sequences from 1680 sequences
kept 1593 sequences from 1680 sequences
kept 1593 sequences from 1593 sequences
kept 1593 sequences from 1593 sequences
kept 41 sequences from 184 sequences
kept 41 sequences from 184 sequences
kept 34 sequences from 41 sequences
kept 34 sequences from 41 sequences


In [3]:
data_manager.train_seq2seq.seq_x.texts[-2]

'il me faut du temps . <eos>'

In [4]:
data_manager.train_seq2seq.seq_x.toks_id[-2]

[13, 21, 59, 57, 134, 4, 3]

In [5]:
data_manager.train_seq2seq.seq_y.texts[-2]

'i need time . <eos>'

In [6]:
data_manager.train_seq2seq.seq_y.toks_id[-2]

[5, 54, 122, 4, 3]

In [7]:
data_manager.train_seq2seq.seq_x.vocab.stoi

defaultdict(int,
            {'<unk>': 0,
             '<pad>': 1,
             '<bos>': 2,
             '<eos>': 3,
             '.': 4,
             '!': 5,
             'je': 6,
             'suis': 7,
             'est': 8,
             '?': 9,
             'j': 10,
             'ai': 11,
             'nous': 12,
             'il': 13,
             'le': 14,
             'vous': 15,
             'en': 16,
             'a': 17,
             'c': 18,
             'moi': 19,
             'pas': 20,
             'me': 21,
             'l': 22,
             'de': 23,
             'tom': 24,
             'ca': 25,
             'soyez': 26,
             'ne': 27,
             'la': 28,
             'toi': 29,
             'qui': 30,
             'un': 31,
             'ce': 32,
             'avons': 33,
             'sois': 34,
             'bien': 35,
             'y': 36,
             'tu': 37,
             'm': 38,
             'aller': 39,
             't': 40,
             'va': 41,


In [8]:
data_manager.valid_seq2seq.seq_x.texts[-2]

'il m en faut davantage . <eos>'

In [9]:
data_manager.valid_seq2seq.seq_y.texts[-2]

'i need more . <eos>'

## Train

In [10]:

fit(25, encoder, encoder_optimizer, decoder, decoder_optimizer, train_batch_size, valid_batch_size, clip,
    masked_cross_entropy, train_dl=train_dataloader, valid_dl=valid_dataloader)



0:0:29.65 (1 4.0%) loss train: 3.961 loss valid: 3.777
0:0:25.35 (2 8.0%) loss train: 2.904 loss valid: 3.733
0:0:22.44 (3 12.0%) loss train: 2.506 loss valid: 3.584
0:0:23.02 (4 16.0%) loss train: 2.279 loss valid: 3.044
0:0:23.7 (5 20.0%) loss train: 2.158 loss valid: 2.928
0:0:33.14 (6 24.0%) loss train: 2.07 loss valid: 2.807
0:0:34.32 (7 28.0%) loss train: 2.04 loss valid: 2.558
0:0:28.27 (8 32.0%) loss train: 1.885 loss valid: 3.042
0:0:19.31 (9 36.0%) loss train: 1.734 loss valid: 2.564
0:0:23.05 (10 40.0%) loss train: 1.645 loss valid: 2.733
0:0:24.61 (11 44.0%) loss train: 1.544 loss valid: 2.566
0:0:21.28 (12 48.0%) loss train: 1.465 loss valid: 2.397
0:0:24.39 (13 52.0%) loss train: 1.391 loss valid: 2.209
0:0:25.01 (14 56.0%) loss train: 1.313 loss valid: 2.01
0:0:25.41 (15 60.0%) loss train: 1.249 loss valid: 2.044
0:0:28.54 (16 64.0%) loss train: 1.192 loss valid: 2.013
0:0:34.06 (17 68.0%) loss train: 1.141 loss valid: 2.005
0:0:25.7 (18 72.0%) loss train: 1.09 loss vali

In [11]:
#predicted_text=predict('i loved you.', encoder, decoder, data_manager)
original_xtext='Je suis sûr.'
original_ytext='I am sure.'
predicted_text=predict(original_xtext, encoder, decoder, data_manager)
print(f'original text: {original_xtext}')
print(f'original answer: {original_ytext}')
print(f'predicted text: {predicted_text}')

original text: Je suis sûr.
original answer: I am sure.
predicted text: am <unk> . <eos>


In [12]:
data_manager.train_seq2seq.seq_y.vocab.stoi['sure']

0

In [13]:
data_manager.train_seq2seq.seq_x.vocab.stoi

defaultdict(int,
            {'<unk>': 0,
             '<pad>': 1,
             '<bos>': 2,
             '<eos>': 3,
             '.': 4,
             '!': 5,
             'je': 6,
             'suis': 7,
             'est': 8,
             '?': 9,
             'j': 10,
             'ai': 11,
             'nous': 12,
             'il': 13,
             'le': 14,
             'vous': 15,
             'en': 16,
             'a': 17,
             'c': 18,
             'moi': 19,
             'pas': 20,
             'me': 21,
             'l': 22,
             'de': 23,
             'tom': 24,
             'ca': 25,
             'soyez': 26,
             'ne': 27,
             'la': 28,
             'toi': 29,
             'qui': 30,
             'un': 31,
             'ce': 32,
             'avons': 33,
             'sois': 34,
             'bien': 35,
             'y': 36,
             'tu': 37,
             'm': 38,
             'aller': 39,
             't': 40,
             'va': 41,


In [14]:
#set([for tok in toks data_manager.train_seq2seq.seq_x.toks])
len(set([item for sublist in data_manager.train_seq2seq.seq_x.toks for item in sublist]))

1077

In [15]:
len(data_manager.train_seq2seq.seq_x.vocab.stoi)

1080

In [16]:
len(data_manager.train_seq2seq.seq_x.vocab.itos)

163

In [17]:
data_manager.train_seq2seq.seq_x.vocab.itos

['<unk>',
 '<pad>',
 '<bos>',
 '<eos>',
 '.',
 '!',
 'je',
 'suis',
 'est',
 '?',
 'j',
 'ai',
 'nous',
 'il',
 'le',
 'vous',
 'en',
 'a',
 'c',
 'moi',
 'pas',
 'me',
 'l',
 'de',
 'tom',
 'ca',
 'soyez',
 'ne',
 'la',
 'toi',
 'qui',
 'un',
 'ce',
 'avons',
 'sois',
 'bien',
 'y',
 'tu',
 'm',
 'aller',
 't',
 'va',
 'les',
 'laisse',
 'laissez',
 'sommes',
 'une',
 'aime',
 'peux',
 'n',
 'd',
 ',',
 'fait',
 'es',
 'fais',
 'que',
 'ici',
 'du',
 'elle',
 'faut',
 'vu',
 'partir',
 's',
 'etes',
 'ete',
 'reste',
 'chez',
 'sens',
 'sais',
 'prenez',
 'allez',
 'tomber',
 'te',
 'malade',
 'prends',
 'deteste',
 'essaye',
 'perdu',
 'vais',
 'on',
 'comme',
 'elles',
 'faites',
 'ceci',
 'pouvons',
 'comment',
 'besoin',
 'restez',
 'gagne',
 'calme',
 'tenez',
 'plus',
 'nouveau',
 'ils',
 'apprecie',
 'arrete',
 'attends',
 'bon',
 'demande',
 'venez',
 'faire',
 'aide',
 'puis',
 'parle',
 'maison',
 'mis',
 'veux',
 'train',
 'viens',
 'pars',
 'voir',
 'sur',
 'accord',
 'qu'

In [18]:
b=next(iter(train_dataloader))

In [19]:
b[0]

tensor([[ 13,   0,  18,   6,  12,  41,  17,  10,   6,   6,   6,   6,  10,   6,
          16,  18,  16,   0,  12,  12,  98,  43,  44,   0, 109,  70,  17,  25,
         135,  25,  95,   6,  10,   0,   0,   6,  25,   0,  79,  34,  26,  26,
          34,  26,  26,  34,  26,  26,  34,  34,  34,  26,  26,  26,  26, 138,
           0, 138,   0,  43,  44,  41,  70,  41,   0,   0,   0,  96, 117,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0, 137,   0,   0,   0, 139,
           0,   0, 139,  70,  70, 108,  99,   0,   0,   0,   0,   0, 109, 109,
           0,   0],
        [ 49,  19,   8,  22,  22,  72,   0,  11,   7,   7,   7,   7,  11,  78,
           0,   8,   0,  23,  33,  22,  17,  14,  14,  14,  50,  15,  28,   0,
           0,   0,  29,   0,  76,   0,  29,  68,  41,  97,  76,  89,  89,   0,
           0,   0,   0,   0,   0,   0, 119, 119,   0, 119,   0,   0,   0,  19,
          19,  12,  12,  71,  71,   0,   0,   5,   5,   4,   5,   5,   5,   5,
           5,   5,   5,   5,   9

In [36]:
import pandas as pd

In [39]:
np.array(b[0])

array([[ 13,   0,  18,   6,  12,  41,  17,  10,   6,   6,   6,   6,  10,
          6,  16,  18,  16,   0,  12,  12,  98,  43,  44,   0, 109,  70,
         17,  25, 135,  25,  95,   6,  10,   0,   0,   6,  25,   0,  79,
         34,  26,  26,  34,  26,  26,  34,  26,  26,  34,  34,  34,  26,
         26,  26,  26, 138,   0, 138,   0,  43,  44,  41,  70,  41,   0,
          0,   0,  96, 117,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0, 137,   0,   0,   0, 139,   0,   0, 139,  70,  70, 108,  99,
          0,   0,   0,   0,   0, 109, 109,   0,   0],
       [ 49,  19,   8,  22,  22,  72,   0,  11,   7,   7,   7,   7,  11,
         78,   0,   8,   0,  23,  33,  22,  17,  14,  14,  14,  50,  15,
         28,   0,   0,   0,  29,   0,  76,   0,  29,  68,  41,  97,  76,
         89,  89,   0,   0,   0,   0,   0,   0,   0, 119, 119,   0, 119,
          0,   0,   0,  19,  19,  12,  12,  71,  71,   0,   0,   5,   5,
          4,   5,   5,   5,   5,   5,   5,   5,   5,   9,   4,   4,   